In [125]:
from collections import defaultdict

next_trigger_time = None
order_book = {}
vwap = {}
report_book = defaultdict(lambda: {})

In [126]:
# Initialise constants

events_length = {
    "S": 12,
    "R": 39,
    "H": 25,
    "Y": 20,
    "L": 26,
    "V": 35,
    "W": 12,
    "K": 28,
    "J": 35,
    "h": 21,
    "A": 36, # Add order
    "F": 40, # Add order
    "E": 31, # Order executed
    "C": 36, # Order executed with price
    "X": 23,
    "D": 19,
    "U": 35, # Order replace msg
    "P": 44, # Trade msg
    "Q": 40, # Trade msg
    "B": 19,
    "I": 50,
    "N": 20,
    "O": 48 
}

msg_formats = {
    "S": {  # Order executed
        "format_str": ">HH6sc",
        "time_stmp": 2,
        "event_code": 3
    }, 
    "E": {  # Order executed
        "format_str": ">HH6sQIQ",
        "time_stmp": 2,
        "ord_ref_num": 3,
        "shares": 4
    },
    "C": {  # Order executed with price
        "format_str": ">HH6sQIQcI", 
        "time_stmp": 2,
        "ord_ref_num": 3,
        "shares": 4,
        "printable": 6,
        "price": 7
    },
    "P": {  # Non cross for hidden orders
        "format_str": ">HH6sQcI8sIQ",
        "time_stmp": 2,
        "shares": 5,
        "stock": 6,
        "price": 7
    },
    "Q": {  # Cross trades
        "format_str": ">HH6sQ8sIQc",
        "time_stmp": 2,
        "shares": 3,
        "stock": 4,
        "price": 5
    },
    "A": {  # Add Order
        "format_str": ">HH6sQcI8sI",
        "time_stmp": 2,
        "ord_ref_num": 3,
        "shares": 5,
        "stock": 6,
        "price": 7
    },
    "F": {  # Add Order with MPID
        "format_str": ">HH6sQcI8sI4s",
        "time_stmp": 2,
        "ord_ref_num": 3,
        "shares": 5,
        "stock": 6,
        "price": 7
    },
    "U": {  # Replace Order
        "format_str": ">HH6sQQII",
        "time_stmp": 2,
        "old_ord_ref_num": 3,
        "ord_ref_num": 4,
        "shares": 5,
        "price": 6
    },
    "D": {  # Delete Order
        "format_str": ">HH6sQ",
        "time_stmp": 2,
        "ord_ref_num": 3
    },
}

NS_PER_HOUR = 60*60*10**9

In [127]:
# Defined functions for calculating VWAP and Updating order book 

import struct

def calc_vwap(msg_type, data):
    global next_trigger_time
    
    value = struct.unpack(msg_formats[msg_type]["format_str"], data)
    time = struct.unpack(">Q", b"\x00\x00"+value[msg_formats[msg_type]["time_stmp"]])[0]
    if next_trigger_time and time >= next_trigger_time:
        report_vwap(next_trigger_time)
        next_trigger_time += timedelta(hours=1) 

    shares = value[msg_formats[msg_type]["shares"]]
    if shares == 0:
        """According to section 1.5.2 of ITCH specification if Cross cannot be 
        conducted then shares is returned as 0"""
        return None
        
    if "ord_ref_num" in msg_formats[msg_type]:
        ord_ref_num = value[msg_formats[msg_type]["ord_ref_num"]]

    if "stock" in msg_formats[msg_type]:
        stock = value[msg_formats[msg_type]["stock"]].decode('ascii').strip()
    else:
        stock = order_book[ord_ref_num]["stock"]

    if "price" in msg_formats[msg_type]:
        price = value[msg_formats[msg_type]["price"]]/10000
    else:
        price = order_book[ord_ref_num]["price"]     

    
    if msg_type in {"E", "C"}:
        order_book[ord_ref_num]["shares"] -= shares
        if order_book[ord_ref_num]["shares"] == 0:
            del order_book[ord_ref_num]
    
    if msg_type == "C" and value[msg_formats[msg_type]["printable"]].decode('ascii')=="N":
        """According to section 1.4.2 of ITCH specification non-printable
        should be ignored for volume calculations"""
        return None

    
    if stock in vwap:
        vwap[stock]["PV"] += price*shares
        vwap[stock]["V"] += shares
    else:
        vwap[stock] = {
            "PV": price*shares , 
            "V": shares
        } 
        

def update_order_book(msg_type, data):
    global next_trigger_time
    
    value = struct.unpack(msg_formats[msg_type]["format_str"], data)
    time = struct.unpack(">Q", b"\x00\x00"+value[msg_formats[msg_type]["time_stmp"]])[0]
    if next_trigger_time and time >= next_trigger_time:
        report_vwap(next_trigger_time)
        next_trigger_time += NS_PER_HOUR
        
    ord_ref_num = value[msg_formats[msg_type]["ord_ref_num"]]

    if msg_type == "D":
        del order_book[ord_ref_num]
        return None

    if "stock" in msg_formats[msg_type]:
        stock = value[msg_formats[msg_type]["stock"]].decode('ascii').strip()
    else:
        old_ord_ref_num = value[msg_formats[msg_type]["old_ord_ref_num"]]
        stock = order_book[old_ord_ref_num]["stock"]
        del order_book[old_ord_ref_num]

    
    shares = value[msg_formats[msg_type]["shares"]]
    price = value[msg_formats[msg_type]["price"]]/10000

    order_book[ord_ref_num] = {
        "stock": stock,
        "price": price,
        "shares": shares
    }


In [128]:
from datetime import time as dt_t, timedelta

def nanoseconds_to_time(nanoseconds):
    NS_PER_SECOND = 10**9
    SECONDS_PER_HOUR = 60*60
    SECONDS_PER_MINUTE = 60
    
    total_seconds, remaining_ns = divmod(nanoseconds, NS_PER_SECOND)
    
    hours, remainder = divmod(total_seconds, SECONDS_PER_HOUR)
    minutes, seconds = divmod(remainder, SECONDS_PER_MINUTE)
    
    microseconds = remaining_ns // 1000  # 1 microsecond = 1,000 nanoseconds
    
    return dt_t(hour=int(hours), minute=int(minutes), second=int(seconds), microsecond=int(microseconds))

def display_system_event(msg_type, data):
    global next_trigger_time
    
    value = struct.unpack(msg_formats[msg_type]["format_str"], data)
    time = struct.unpack(">Q", b"\x00\x00"+value[msg_formats[msg_type]["time_stmp"]])[0]
    event_code = value[msg_formats[msg_type]["event_code"]].decode('ascii')
    if event_code == "Q":
        next_trigger_time = time + NS_PER_HOUR 
        report_vwap(time)
    elif event_code == "M":
        next_trigger_time = None 
        report_vwap(time)
    print(f"===== System Message {event_code} {nanoseconds_to_time(time)} =====")

def report_vwap(time):
    global report_book
    
    time = nanoseconds_to_time(time)
    print(f"Generating report at {time}")
    for stock in vwap:
        report_book[stock][time] = vwap[stock]["PV"]/vwap[stock]["V"]
    

In [129]:
# Run main code

from sys import getsizeof
import time

t0 = time.time()
with open('../01302019_NASDAQ_ITCH50', mode="rb") as file:
    count = 0
    while True:
        file.seek(2, 1)
        data = file.read(1)
        msg_type = data.decode('ascii')

        if msg_type=="":
            print("\n++++++ Reached EOF ++++++")
            break
       
        # if count == 2e6:
        #     print(msg_type)
        #     report_vwap(9*NS_PER_HOUR)
        #     print(f"END")
        #     break

        
        if count % 10000000 == 0:
            print(msg_type)
            print(f"order book, {len(order_book):_}, {getsizeof(order_book):_}")
            print(f"vwap, {len(vwap):_}, {getsizeof(vwap):_}")
        
        count+=1
        if msg_type == "S":
            data2 = file.read(events_length[msg_type]-1)
            display_system_event(msg_type, data2)
        elif msg_type in {"E", "C", "P", "Q"}:
            data2 = file.read(events_length[msg_type]-1)
            calc_vwap(msg_type, data2)
        elif msg_type in {"A", "F", "U", "D"}:
            data2 = file.read(events_length[msg_type]-1)
            update_order_book(msg_type, data2)    
        else:
            file.seek(events_length[msg_type]-1, 1)

t1=time.time()
print(f"\n++++++ Time elasped: {t1-t0:_} sec ++++++")

S
order book, 0, 64
vwap, 0, 64
===== System Message O 03:03:59.687760 =====
===== System Message S 04:00:00.000181 =====
I
order book, 341_133, 20_971_600
vwap, 943, 26_032
Generating report at 09:30:00.000036
===== System Message Q 09:30:00.000036 =====
A
order book, 1_534_346, 167_772_248
vwap, 4_943, 103_856
D
order book, 1_550_363, 167_772_248
vwap, 5_404, 103_856
A
order book, 1_588_572, 167_772_248
vwap, 5_623, 207_616
A
order book, 1_597_338, 167_772_248
vwap, 5_819, 207_616
A
order book, 1_610_294, 167_772_248
vwap, 5_979, 207_616
D
order book, 1_620_179, 167_772_248
vwap, 6_095, 207_616
Generating report at 10:30:00.000036
A
order book, 1_620_147, 167_772_248
vwap, 6_235, 207_616
D
order book, 1_634_257, 167_772_248
vwap, 6_356, 207_616
U
order book, 1_637_653, 167_772_248
vwap, 6_447, 207_616
U
order book, 1_637_399, 167_772_248
vwap, 6_538, 207_616
U
order book, 1_648_515, 167_772_248
vwap, 6_602, 207_616
D
order book, 1_651_108, 167_772_248
vwap, 6_682, 207_616
Generating 

In [131]:
order_book

{86786: {'stock': 'ING', 'price': 12.07, 'shares': 600},
 148981: {'stock': 'AU', 'price': 13.89, 'shares': 100},
 91140: {'stock': 'STM', 'price': 16.05, 'shares': 1900},
 132627: {'stock': 'SAP', 'price': 102.2, 'shares': 792},
 132643: {'stock': 'SAP', 'price': 102.19, 'shares': 333},
 132831: {'stock': 'SAP', 'price': 102.2, 'shares': 90},
 163097: {'stock': 'BABA', 'price': 158.84, 'shares': 5},
 103864: {'stock': 'TVIX', 'price': 46.48, 'shares': 100},
 161639: {'stock': 'NOK', 'price': 6.46, 'shares': 2800},
 120400: {'stock': 'TVIX', 'price': 46.42, 'shares': 100},
 121328: {'stock': 'TVIX', 'price': 46.42, 'shares': 100},
 254881: {'stock': 'BABA', 'price': 160.0, 'shares': 10},
 151116: {'stock': 'TVIX', 'price': 44.0, 'shares': 50},
 300509: {'stock': 'ACB', 'price': 6.94, 'shares': 2000},
 264123: {'stock': 'MT', 'price': 23.35, 'shares': 600},
 276547: {'stock': 'NVO', 'price': 46.3, 'shares': 400},
 398181: {'stock': 'CHEK', 'price': 3.89, 'shares': 1422},
 432649: {'stoc

In [115]:
for k,v in order_book.items():
    if v['stock'] == "JPBI":
        print(k, v)

6096682 {'stock': 'JPBI', 'price': 15.55, 'shares': 100}
6096686 {'stock': 'JPBI', 'price': 27.85, 'shares': 100}
6097194 {'stock': 'JPBI', 'price': 20.18, 'shares': 100}
6097218 {'stock': 'JPBI', 'price': 23.16, 'shares': 100}
6097226 {'stock': 'JPBI', 'price': 20.72, 'shares': 100}
6097238 {'stock': 'JPBI', 'price': 22.61, 'shares': 100}
6332314 {'stock': 'JPBI', 'price': 27.82, 'shares': 100}
6332318 {'stock': 'JPBI', 'price': 15.55, 'shares': 100}
6332386 {'stock': 'JPBI', 'price': 27.82, 'shares': 100}
6332390 {'stock': 'JPBI', 'price': 15.55, 'shares': 100}
6342966 {'stock': 'JPBI', 'price': 16.2, 'shares': 100}
6342970 {'stock': 'JPBI', 'price': 18.79, 'shares': 100}
6342982 {'stock': 'JPBI', 'price': 20.73, 'shares': 100}
6342986 {'stock': 'JPBI', 'price': 24.56, 'shares': 100}
6343002 {'stock': 'JPBI', 'price': 27.17, 'shares': 100}
6391118 {'stock': 'JPBI', 'price': 16.2, 'shares': 100}
6391202 {'stock': 'JPBI', 'price': 27.17, 'shares': 100}
6575930 {'stock': 'JPBI', 'price'

In [132]:
vwap

{'XLV': {'PV': 211754168.23999915, 'V': 2378065},
 'TVIX': {'PV': 148590871.62000084, 'V': 3315087},
 'AAPL': {'PV': 2374162743.47502, 'V': 14515529},
 'DPW': {'PV': 37707.194800000005, 'V': 406915},
 'UGAZ': {'PV': 35626923.585000046, 'V': 922187},
 'HMY': {'PV': 778212.52, 'V': 398588},
 'GOLD': {'PV': 22044120.79999992, 'V': 1696227},
 'DRD': {'PV': 29408.61, 'V': 13460},
 'QCOM': {'PV': 270387522.98999995, 'V': 5403884},
 'SPY': {'PV': 4956124728.079975, 'V': 18596334},
 'UPRO': {'PV': 41177930.435, 'V': 989017},
 'AMD': {'PV': 887216970.7500011, 'V': 39866581},
 'TEF': {'PV': 1772342.9200000002, 'V': 203815},
 'BX': {'PV': 23815655.760000028, 'V': 725381},
 'FB': {'PV': 1844045783.2899954, 'V': 12054147},
 'AMZN': {'PV': 2980297732.475007, 'V': 1802474},
 'GOOG': {'PV': 540689659.5200012, 'V': 499566},
 'ALGN': {'PV': 272919895.92500114, 'V': 1218520},
 'OCX': {'PV': 3362258.1650000014, 'V': 639361},
 'AU': {'PV': 8807129.935000006, 'V': 640791},
 'SAP': {'PV': 23479612.309999976,

In [133]:
sorted(vwap.items(), key=lambda x: x[1]["V"])

[('TZAC', {'PV': 10.47, 'V': 1}),
 ('UNL', {'PV': 11.01, 'V': 1}),
 ('LNGR', {'PV': 20.58, 'V': 1}),
 ('FTXH', {'PV': 19.9, 'V': 1}),
 ('FMCIU', {'PV': 10.19, 'V': 1}),
 ('JDIV', {'PV': 24.92, 'V': 1}),
 ('FLGE', {'PV': 216.0, 'V': 1}),
 ('CHNA', {'PV': 20.58, 'V': 1}),
 ('FORTY', {'PV': 39.04, 'V': 1}),
 ('CRBN', {'PV': 110.53, 'V': 1}),
 ('REDV', {'PV': 24.27, 'V': 1}),
 ('HEWU', {'PV': 22.18, 'V': 1}),
 ('WFHY', {'PV': 49.4, 'V': 1}),
 ('ONEO', {'PV': 67.82, 'V': 1}),
 ('JJT', {'PV': 52.27, 'V': 1}),
 ('USVM', {'PV': 49.6, 'V': 1}),
 ('ZDGE', {'PV': 1.94, 'V': 1}),
 ('PHDG', {'PV': 26.59, 'V': 1}),
 ('RDIB', {'PV': 29.39, 'V': 1}),
 ('YLDE', {'PV': 27.49, 'V': 1}),
 ('RIVE', {'PV': 12.6, 'V': 1}),
 ('SUSC', {'PV': 24.3, 'V': 1}),
 ('AHL-D', {'PV': 23.13, 'V': 1}),
 ('LOWC', {'PV': 84.94, 'V': 1}),
 ('IDHD', {'PV': 26.95, 'V': 1}),
 ('LMHA', {'PV': 26.39, 'V': 1}),
 ('FTV-A', {'PV': 977.53, 'V': 1}),
 ('IKNX', {'PV': 8.745, 'V': 1}),
 ('BFOR', {'PV': 39.16, 'V': 1}),
 ('VFLQ', {'PV':

In [134]:
len(vwap)

7538

In [135]:
report_book

defaultdict(<function __main__.<lambda>()>,
            {'XLV': {datetime.time(9, 30, 0, 36): 88.10375,
              datetime.time(10, 30, 0, 36): 88.72463528500816,
              datetime.time(11, 30, 0, 36): 88.70385311414582,
              datetime.time(12, 30, 0, 36): 88.77564978525629,
              datetime.time(13, 30, 0, 36): 88.7947777332637,
              datetime.time(14, 30, 0, 36): 88.90226043841088,
              datetime.time(15, 30, 0, 36): 88.98444629330196,
              datetime.time(16, 0, 0, 113): 89.0445927486941},
             'TVIX': {datetime.time(9, 30, 0, 36): 45.95085728462916,
              datetime.time(10, 30, 0, 36): 46.25955605812278,
              datetime.time(11, 30, 0, 36): 46.17583436069051,
              datetime.time(12, 30, 0, 36): 46.04612225063713,
              datetime.time(13, 30, 0, 36): 45.961343550442905,
              datetime.time(14, 30, 0, 36): 45.46287552048745,
              datetime.time(15, 30, 0, 36): 45.070422749632066,
      

In [136]:
import pandas as pd

df = pd.DataFrame(report_book).T
print(df)

      09:30:00.000036  10:30:00.000036  11:30:00.000036  12:30:00.000036  \
XLV         88.103750        88.724635        88.703853        88.775650   
TVIX        45.950857        46.259556        46.175834        46.046122   
AAPL       162.520038       162.081871       162.190693       162.192105   
DPW          0.094668         0.093719         0.093449         0.092911   
UGAZ        38.769293        38.587710        38.622480        38.750595   
...               ...              ...              ...              ...   
FCO               NaN              NaN              NaN              NaN   
LEJU              NaN              NaN              NaN              NaN   
YAO               NaN              NaN              NaN              NaN   
OBAS              NaN              NaN              NaN              NaN   
TAPR              NaN              NaN              NaN              NaN   

      13:30:00.000036  14:30:00.000036  15:30:00.000036  16:00:00.000113  
XLV         

In [137]:
df.to_csv('output.csv')

# Sandbox

### VWAP Analysis

In [16]:
vwap

{'XLV     ': {'PV': 211754168.23999915, 'V': 2378065},
 'TVIX    ': {'PV': 148590871.62000084, 'V': 3315087},
 'AAPL    ': {'PV': 2374162743.47502, 'V': 14515529},
 'DPW     ': {'PV': 37707.194800000005, 'V': 406915},
 'UGAZ    ': {'PV': 35626923.585000046, 'V': 922187},
 'HMY     ': {'PV': 778212.52, 'V': 398588},
 'GOLD    ': {'PV': 22044120.79999992, 'V': 1696227},
 'DRD     ': {'PV': 29408.61, 'V': 13460},
 'QCOM    ': {'PV': 270387522.98999995, 'V': 5403884},
 'SPY     ': {'PV': 4956124728.079975, 'V': 18596334},
 'UPRO    ': {'PV': 41177930.435, 'V': 989017},
 'AMD     ': {'PV': 887216970.7500011, 'V': 39866581},
 'TEF     ': {'PV': 1772342.9200000002, 'V': 203815},
 'BX      ': {'PV': 23815655.760000028, 'V': 725381},
 'FB      ': {'PV': 1844045783.2899954, 'V': 12054147},
 'AMZN    ': {'PV': 2980297732.475007, 'V': 1802474},
 'GOOG    ': {'PV': 540689659.5200012, 'V': 499566},
 'ALGN    ': {'PV': 272919895.92500114, 'V': 1218520},
 'OCX     ': {'PV': 3362258.1650000014, 'V': 63

In [17]:
len(vwap.keys())

8713

In [18]:
sorted(vwap.items(), key=lambda x: x[1]['V'], reverse=True) 

[('AMD     ', {'PV': 887216970.7500011, 'V': 39866581}),
 ('HMNY    ', {'PV': 235072.89759999537, 'V': 21136911}),
 ('SPY     ', {'PV': 4956124728.079975, 'V': 18596334}),
 ('VALE    ', {'PV': 227539450.68000004, 'V': 18251831}),
 ('EEM     ', {'PV': 627856224.1599989, 'V': 14797898}),
 ('AAPL    ', {'PV': 2374162743.47502, 'V': 14515529}),
 ('MSFT    ', {'PV': 1406069485.1449919, 'V': 13347965}),
 ('SIRI    ', {'PV': 74316784.67999983, 'V': 12540032}),
 ('FB      ', {'PV': 1844045783.2899954, 'V': 12054147}),
 ('CZR     ', {'PV': 107218176.64499982, 'V': 11928705}),
 ('T       ', {'PV': 294457183.0300002, 'V': 10031383}),
 ('QQQ     ', {'PV': 1616273808.4999976, 'V': 9803170}),
 ('GE      ', {'PV': 87443838.27499999, 'V': 9584381}),
 ('XLF     ', {'PV': 226054602.1200001, 'V': 8707144}),
 ('BAC     ', {'PV': 251752882.01999998, 'V': 8606374}),
 ('GDX     ', {'PV': 189603172.19000006, 'V': 8571637}),
 ('INTC    ', {'PV': 400552842.2349996, 'V': 8494587}),
 ('CODX    ', {'PV': 24208549.

In [19]:
sum([i['V'] for i in vwap.values()])  # Total volumme of shares traded

1408908286

In [20]:
39866581/1408908286

0.0282960795930758

### Need to consider Delete Order events as it will reduce size of order book

In [23]:
from collections import defaultdict

count = defaultdict(int)
with open('../01302019_NASDAQ_ITCH50', mode="rb") as file:
    cnt = 0
    last_pos = 0
    while True:
        file.seek(2, 1)
        data = file.read(1)
        msg_type = data.decode('ascii')
        cnt += 1
        if cnt%10000000 == 0:
            print(count)
        
        if msg_type=="":
            print("Reached EOF")
            break
        # print(msg_type)
        count[msg_type]+=1
        file.seek(events_length[msg_type]-1, 1)


defaultdict(<class 'int'>, {'S': 2, 'R': 8714, 'H': 8716, 'Y': 8713, 'L': 193722, 'A': 3666936, 'D': 3545789, 'F': 245915, 'U': 449014, 'X': 1067667, 'E': 42671, 'P': 13021, 'V': 1, 'I': 749117, 'C': 1})
defaultdict(<class 'int'>, {'S': 3, 'R': 8714, 'H': 8716, 'Y': 8727, 'L': 193724, 'A': 8014012, 'D': 7071154, 'F': 778592, 'U': 1368941, 'X': 1187389, 'E': 239240, 'P': 55566, 'V': 1, 'I': 1051216, 'C': 5291, 'Q': 8713})
defaultdict(<class 'int'>, {'S': 3, 'R': 8714, 'H': 8717, 'Y': 8732, 'L': 193724, 'A': 12460682, 'D': 11402682, 'F': 839066, 'U': 2160667, 'X': 1316842, 'E': 434650, 'P': 95514, 'V': 1, 'I': 1051216, 'C': 10076, 'Q': 8713})
defaultdict(<class 'int'>, {'S': 3, 'R': 8714, 'H': 8721, 'Y': 8737, 'L': 193726, 'A': 16890529, 'D': 15686597, 'F': 881185, 'U': 3063336, 'X': 1433016, 'E': 618676, 'P': 131922, 'V': 1, 'I': 1051816, 'C': 14305, 'Q': 8713, 'J': 2})
defaultdict(<class 'int'>, {'S': 3, 'R': 8714, 'H': 8722, 'Y': 8743, 'L': 193727, 'A': 21354416, 'D': 20014437, 'F': 9

In [24]:
# Data

from pprint import pprint

pprint(count)

defaultdict(<class 'int'>,
            {'A': 162970455,
             'B': 116,
             'C': 158886,
             'D': 158273361,
             'E': 8096995,
             'F': 1725898,
             'H': 8805,
             'I': 3684511,
             'J': 62,
             'L': 193769,
             'P': 1326184,
             'Q': 17430,
             'R': 8714,
             'S': 6,
             'U': 27222746,
             'V': 1,
             'X': 4669874,
             'Y': 8821})


In [ ]:
# Total number of events
sum(count.values())

In [26]:
sorted(count.items(), key=lambda x: x[1], reverse=True)

[('A', 162970455),
 ('D', 158273361),
 ('U', 27222746),
 ('E', 8096995),
 ('X', 4669874),
 ('I', 3684511),
 ('F', 1725898),
 ('P', 1326184),
 ('L', 193769),
 ('C', 158886),
 ('Q', 17430),
 ('Y', 8821),
 ('H', 8805),
 ('R', 8714),
 ('B', 116),
 ('J', 62),
 ('S', 6),
 ('V', 1)]

This shows the number of deleted orders are of the same magnitude as added orders and hence can be considered in order to keep the order book lean